In [1]:
import pandas as pd
import opendatasets as od
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

2023-11-20 05:04:45.665196: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 05:04:47.843120: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 05:04:47.843219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 05:04:48.156086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 05:04:48.858671: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 05:04:48.859859: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
od.download("https://www.kaggle.com/datasets/alexteboul/heart-disease-health-indicators-dataset")

100%|██████████| 2.66M/2.66M [00:00<00:00, 54.3MB/s]

In [3]:
!mv "./heart-disease-health-indicators-dataset/heart_disease_health_indicators_BRFSS2015.csv" "../data/raw/df_raw.csv"

In [4]:
df_raw = pd.read_csv("../data/raw/df_raw.csv")
df_raw.head(5)

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [5]:
df_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
HeartDiseaseorAttack,253680.0,0.094186,0.292087,0.0,0.0,0.0,0.0,1.0
HighBP,253680.0,0.429001,0.494934,0.0,0.0,0.0,1.0,1.0
HighChol,253680.0,0.424121,0.494210,0.0,0.0,0.0,1.0,1.0
CholCheck,253680.0,0.962670,0.189571,0.0,1.0,1.0,1.0,1.0
BMI,253680.0,28.382364,6.608694,12.0,24.0,27.0,31.0,98.0
Smoker,253680.0,0.443169,0.496761,0.0,0.0,0.0,1.0,1.0
Stroke,253680.0,0.040571,0.197294,0.0,0.0,0.0,0.0,1.0
Diabetes,253680.0,0.296921,0.698160,0.0,0.0,0.0,0.0,2.0
PhysActivity,253680.0,0.756544,0.429169,0.0,1.0,1.0,1.0,1.0
Fruits,253680.0,0.634256,0.481639,0.0,0.0,1.0,1.0,1.0


In [6]:
df_raw = df_raw.drop(["CholCheck", "Fruits", "Veggies", "AnyHealthcare", "NoDocbcCost", "GenHlth", "MentHlth", "PhysHlth", "DiffWalk", "Age", "Education", "Income"], axis=1).copy()

In [7]:
X = df_raw.drop(['HeartDiseaseorAttack'], axis=1).copy()
y = df_raw["HeartDiseaseorAttack"].copy()

In [15]:
X.head(5)

,HighBP,HighChol,BMI,Smoker,Stroke,Diabetes,PhysActivity,HvyAlcoholConsump,Sex
0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.0,1.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,27.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,1.0,24.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
6342/6342 [==============================] - 9s 1ms/step - loss: 0.2660 - accuracy: 0.9055 - val_loss: 0.2627 - val_accuracy: 0.9070
Epoch 2/10
6342/6342 [==============================] - 9s 1ms/step - loss: 0.2625 - accuracy: 0.9061 - val_loss: 0.2625 - val_accuracy: 0.9068
Epoch 3/10
6342/6342 [==============================] - 11s 2ms/step - loss: 0.2623 - accuracy: 0.9063 - val_loss: 0.2625 - val_accuracy: 0.9070
Epoch 4/10
6342/6342 [==============================] - 7s 1ms/step - loss: 0.2622 - accuracy: 0.9064 - val_loss: 0.2624 - val_accuracy: 0.9068
Epoch 5/10
6342/6342 [==============================] - 7s 1ms/step - loss: 0.2621 - accuracy: 0.9062 - val_loss: 0.2625 - val_accuracy: 0.9072
Epoch 6/10
6342/6342 [==============================] - 7s 1ms/step - loss: 0.2620 - accuracy: 0.9064 - val_loss: 0.2621 - val_accuracy: 0.9069
Epoch 7/10
6342/6342 [==============================] - 8s 1ms/step - loss: 0.2619 - accuracy: 0.9065 - val_loss: 0.2624 - val_accuracy

In [12]:
y_pred = model.predict(X_test)
y_pred_binary = np.round(y_pred)

   1/1586 [..............................] - ETA: 1:18

1586/1586 [==============================] - 1s 839us/step


In [13]:
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Precisión del modelo: {accuracy}')

Precisión del modelo: 0.9071862188584043


In [14]:
joblib.dump(model, '../models/relu_adam_42.pkl')

['../models/relu_adam_42.pkl']